In [ ]:
class MultiLayerPerceptron:
    def __init__(self, inputs_nodes, hidden_nodes, outputs_nodes, learning_rating, bias):
        self.input_nodes = inputs_nodes
        self.hidden_nodes = hidden_nodes
        self.output_nodes = outputs_nodes
        self.lr = learning_rating
        self.bias = bias
        
        np.random.seed()
        
        self.nscale = (2 / self.input_nodes) ** (-0.5)
        self.w12 = np.random.normal(scale = self.nscale, size = (self.input_nodes, self.hidden_nodes))
        self.w23 = np.random.normal(scale = self.nscale, size = (self.hidden_nodes, self.hidden_nodes))
        self.w34 = np.random.normal(scale = self.nscale, size = (self.hidden_nodes, self.output_nodes))
        
        self.b12 = np.random.normal(scale = self.nscale, size = (1, self.hidden_nodes))
        self.b23 = np.random.normal(scale = self.nscale, size = (1, self.hidden_nodes))
        self.b34 = np.random.normal(scale = self.nscale, size = (1, self.output_nodes))
        
        sigmoid = lambda x: 1 / (1 + np.exp(-x))
        deriv_sigmoid = lambda x: (1 / (1 + np.exp(-x)))*(1 - 1 / ((1 + np.exp(-x))))
        
        linear = lambda x: x
        deriv_linear = lambda x: 1
        
        self.act_func = sigmoid
        self.deriv_act_func = deriv_sigmoid
        
        self.act_func_out = linear
        self.deriv_act_func_out = deriv_linear
            
    def shuffle_data(self, features, targets):
        df1 = pd.DataFrame(features)
        df2 = pd.DataFrame(targets)
        df = pd.concat([df1, df2], axis = 1)
        df = df.sample(frac = 1)
    
        features = df.iloc[:, :features.shape[1]].values
        targets = df.iloc[:, features.shape[1]:].values
        return features, targets  
    def zeros(self):
        self.error_rms, self.error_tv_rms = 0, 0
        self.dw12, self.dw23, self.dw34 = np.zeros(self.w12.shape), np.zeros(self.w23.shape), np.zeros(self.w34.shape)
        self.db12, self.db23, self.db34 = np.zeros(self.b12.shape), np.zeros(self.b23.shape), np.zeros(self.b34.shape)
        return None
    def progress(self, e, epochs):
        progress = 100 * e/float(epochs)
        sys.stdout.write("\rProgress: {:2.1f}".format(progress) + "% ")
        sys.stdout.flush()
        return None

    def feedforward(self, x): 
        
        self.phi12 = self.act_func(np.dot(x, self.w12) + (self.bias * self.b12))
        self.dphi12 = self.deriv_act_func(np.dot(x, self.w12) + (self.bias * self.b12))
        
        self.phi23 = self.act_func(np.dot(self.phi12, self.w23) + (self.bias * self.b23))
        self.dphi23 = self.deriv_act_func(np.dot(self.phi12, self.w23) + (self.bias * self.b23))
        
        self.output = self.act_func_out(np.dot(self.phi23, self.w34) + (self.bias * self.b34))
        self.doutput = self.deriv_act_func_out(np.dot(self.phi23, self.w34) + (self.bias * self.b34))

        return None
    def backforward(self):
        
        self.output_error_term = self.error * self.doutput
        self.hidden_error23 = np.dot(self.output_error_term, self.w34.T)
        self.hidden_error_term23 = self.hidden_error23 * self.dphi23
        
        self.hidden_error12 = np.dot(self.hidden_error_term23, self.w23.T)
        self.hidden_error_term12 = self.hidden_error12 * self.dphi12
        
        return None   
    def update_weights(self, x):
        
        self.dw12 += self.hidden_error_term12*x.reshape(self.input_nodes, 1)
        self.dw23 += self.hidden_error_term23*self.phi12.reshape(self.hidden_nodes, 1)
        self.dw34 += self.output_error_term*self.phi23.reshape(self.hidden_nodes, 1)
        
        self.w12 += self.lr*self.dw12/self.input_nodes
        self.w23 += self.lr*self.dw23/self.input_nodes
        self.w34 += self.lr*self.dw34/self.input_nodes
        
        
        self.db12 += self.hidden_error_term12*self.bias
        self.db23 += self.hidden_error_term23*self.bias
        self.db34 += self.output_error_term*self.bias
        
        self.b12 += self.lr*self.db12/self.input_nodes
        self.b23 += self.lr*self.db23/self.input_nodes
        self.b34 += self.lr*self.db34/self.input_nodes
        
        return None

    def run(self, features_test):
        aoutput = []
        for x in features_test:
            self.feedforward(x)
            
            aoutput = np.append(aoutput, self.output)
            
        return aoutput
    
    def train_validation(self, features_train, targets_train, features_test, targets_test, epochs):
        Total_error_vector, Total_error_tv_vector = [], []
        for e in range(epochs):
            features_train, targets_train = self.shuffle_data(features_train, targets_train)
            features_test, targets_test   = self.shuffle_data(features_test,  targets_test)
            self.zeros()
            for x, y in zip(features_train, targets_train):
            
                self.feedforward(x)
            
                self.error = y - self.output
            
                self.backforward()
            
                self.update_weights(x)
                
                self.error_rms += np.mean(self.error**2)
                
            Total_error = self.error_rms/(2 * features_train.shape[0])
            
            Total_error_vector.append(Total_error)
            
            
            for x, y in zip(features_test, targets_test):
                
                self.error_tv = self.run(features_test) - targets_test
                
                self.error_tv_rms += np.mean(self.error_tv**2)
                
            Total_error_tv = self.error_tv_rms/(2 * features_test.shape[0])
            
            Total_error_tv_vector.append(Total_error_tv)
            
            
            self.progress(e, epochs)
            
        return Total_error_vector, Total_error_tv_vector
     

In [ ]:
def split_data(features, targets):
    
        n = np.round(0.7*features.shape[0]).astype(int)
    
        features_train, targets_train = features[:n], targets[:n]
        features_test, targets_test = order_data(features[n:], targets[n:])
    
        return features_train, targets_train, features_test, targets_test
def order_data(features, targets):
        df1 = pd.DataFrame(features)
        df2 = pd.DataFrame(targets)
        df = pd.concat([df1, df2], axis = 1)
        df.columns = ['0', '1']
        df = df.sort_values(by = '0')
        features = df.iloc[:, :features.shape[1]].values
        targets = df.iloc[:, features.shape[1]:].values
    
        return features, targets
def hyper_static(n_features, n_targets):
    hidden_nodes = round((n_features + n_targets)/2) + 1 
    bias = -0.5
    return hidden_nodes, bias
   

def function(features):
    #targets = features**2 + 0.2*features**2 - (features - 0.1)
    #targets = (features**3)+(features**2)-features
    #targets = np.sin(features*6.28)
    targets = features*np.sin(features*2*6.28)
    return targets        

def frd_targets(n_records, n_features):
    features = np.random.rand(n_records, n_features)
    targets = function(features)
    return features, targets
def frd_features_targets(n_records, n_features, n_targets):
        features = np.random.rand(n_records, n_features)
        targets = np.random.rand(n_records, n_targets)
        #features, targets = order_data(features, targets)
        return features, targets

    

def print_trainning_validation(Te, Tev, features_train, targets_train, features_test, aoutput, epochs):
        plt.figure(1)
    
        plt.subplot(121)
        n = np.round(0.001*epochs).astype(int)
        plt.plot(np.array(range(epochs))[n:], Te[n:])
        plt.plot(np.array(range(epochs))[n:], Tev[n:])
        plt.ylim(0, .5)
    
        plt.subplot(122)
        plt.scatter(features_train, targets_train)
        features_test, aoutput = order_data(features_test, aoutput)
        plt.scatter(features_test, aoutput)
        plt.ylim(-1, 1)
        return plt.show()

In [ ]:
import numpy as np
import pandas as pd
import random as rd
import matplotlib.pyplot as plt
import sys


def main_network(learning_rate, epochs):    
    n_records, n_features, n_targets = 30, 1, 1

    features, targets = frd_targets(n_records, n_features)
    features_train, targets_train, features_test, targets_test = split_data(features, targets)
    hidden_nodes, bias = hyper_static(features.shape[1], targets.shape[1])
    
    nn = MultiLayerPerceptron(features.shape[1], hidden_nodes, targets.shape[1], learning_rate, bias)
    
    Te, Tev = nn.train_validation(features_train,targets_train, features_test, targets_test, epochs)
    
    print_trainning_validation(Te, Tev, features_train, targets_train, features_test, nn.run(features_test), epochs)
    return None

def main(loop = 3):
    k = 0.01
    for i in range(0, loop):
        k *= 0.1
        lr, epochs = 0.01, 1000
        main_network(lr, epochs)
        
main(3)